In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["CUDA_DEVICE_ORDEER"] = "PCI_BUS_ID"

In [ ]:
import pickle

import pandas as pd

mentions = pickle.load(open("../data/sentiment_dataset/mentions texts.pickle", "rb"))
mentions.drop(columns=["messageid"], inplace=True)
sentiment = pickle.load(open("../data/sentiment_dataset/sentiment_texts.pickle", "rb"))
len(mentions), len(sentiment)

In [ ]:
issuers = pd.read_excel("../data/sentiment_dataset/names and synonyms.xlsx")
issuers.drop(columns=["VeryOddCompany"], inplace=True)

In [ ]:
issuers["ticker"] = issuers["BGTicker"].copy()
issuers["ticker"].fillna(issuers["BGTicker.1"], inplace=True)
issuers["HumanName"] = issuers["Unnamed: 5"].copy()
issuers["HumanName"].fillna(issuers["Unnamed: 6"], inplace=True)
issuers["HumanName"].fillna(issuers["Unnamed: 7"], inplace=True)
issuers["HumanName"].fillna(issuers["Unnamed: 8"], inplace=True)
issuers["HumanName"].fillna(issuers["Unnamed: 9"], inplace=True)
issuers["HumanName"].fillna(issuers["Unnamed: 10"], inplace=True)
issuers["HumanName"].fillna(issuers["Unnamed: 11"], inplace=True)
issuers["HumanName"].fillna(issuers["Unnamed: 12"], inplace=True)
issuers["HumanName"].fillna(issuers["ticker"], inplace=True)

In [ ]:
merged = pd.merge(
    mentions,
    sentiment,
    left_on=mentions.columns.tolist(),
    right_on=mentions.columns.tolist(),
    how="outer",
)
merged = pd.merge(
    merged,
    issuers[["issuerid", "HumanName", "ticker", "EMITENT_FULL_NAME"]],
    on="issuerid",
)

In [ ]:
merged

In [ ]:
merged["mcid"] = merged["MessageID"].astype(str) + "_" + merged["ChannelID"].astype(str)
merged.iloc[merged["mcid"].drop_duplicates().index]["MessageText"].value_counts()

In [ ]:
print(
    f"Число строк в таблице sentiment: {sentiment.shape[0]}\nЧисло уникальных строк в таблице sentiment: {sentiment['MessageText'].unique().shape[0]}"
)

In [ ]:
unique_sent = sentiment.MessageID.unique()
unique_ment = mentions.MessageID.unique()
print(unique_sent.shape, unique_ment.shape)

In [ ]:
vc = merged.MessageText.value_counts()
vc.min(), vc.median(), vc.mean(), vc.max()

In [ ]:
vc.hist(bins=max(vc))

In [ ]:
vc[vc == 1].shape[0] / vc.shape[0]

In [ ]:
vc.index[1]

In [ ]:
vc.values

In [ ]:
merged.groupby("ChannelID").SentimentScore.mean()

In [ ]:
pd.set_option("display.max_columns", 500)
sentiment.groupby("issuerid")["SentimentScore"].value_counts().unstack(fill_value=0).T

In [ ]:
# import matplotlib.pylab as plt
# plt.figure(figsize=(20,20))
pd.set_option("display.max_rows", 500)
sentiment.groupby("issuerid")["SentimentScore"].value_counts().unstack(fill_value=0)[
    :10
].T.plot(kind="bar", legend=False, figsize=(20, 20))

In [ ]:
s = merged[merged.MessageText == vc.index[7]]
# s.SentimentScore.value_counts()
s

In [ ]:
merged.MessageText

In [ ]:
merged

In [ ]:
merged["found_ticker"] = merged.MessageText.str.findall(r"#[A-Z]{3,5}\s|$[A-Z]{3,5}\s")

In [ ]:
mrg = merged[~merged.MessageText.duplicated(keep="first")]
mrg.shape

In [ ]:
len(mrg[mrg["found_ticker"].str.len() > 0]) / len(mrg)

In [ ]:
import numpy as np

np.intersect1d(unique_ment, unique_sent).shape

In [ ]:
sel_ment = mentions[mentions.MessageID.isin(unique_sent)]
sel_ment.shape

In [ ]:
(mentions.messageid == mentions.MessageID).all()

messageid колонка повторяет MessageID

In [ ]:
mentions.drop(columns=["messageid"], inplace=True)

In [ ]:
sorted(mentions.columns)

In [ ]:
sorted(sentiment.columns)

Разница в колонках таблиц только в новом поле SentimentScore для таблицы sentiment

In [ ]:
usefull_common_cols = ["MessageID", "ChannelID", "MessageText"]

In [ ]:
sentiment[usefull_common_cols]

In [ ]:
import pandas as pd

pd.merge(
    mentions,
    sentiment,
    left_on="MessageID",
)